# 파라메터 서칭
tree base 모델은 설정 가능한 파라메터의 조합에 따라 모델 예측력 차이가 큰 특징을 가지고 있습니다.  
특히, Xgboost 모델의 경우 파라메터 설정에 따른 모델 예측력 차이가 굉장히 크기에 꼭 파라메터 서칭을 진행해주셔야 합니다.  
간단한 문법을 통해 파라메터 서칭을 진행 해보겠습니다.

## 파이썬 기본 문법으로 파라메터 서칭

In [11]:
# 필요 모듈 import
import pandas as pd
from itertools import product
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# 데이터셋 로딩
df = pd.read_csv('./data/boston.csv')
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,y
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
# 타겟 데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [5]:
# 학습 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# 검증 데이터 분할
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [19]:
# 파라메터 서칭
est = [70, 80, 90, 100, 120, 140, 160]
depth = [13, 14, 15, 16, 17, 18]

best_rmse = 999999999
best_param = 0

for param in list(product(est, depth)):
    print(f'{param} 서칭중')
    model = RandomForestRegressor(random_state=42, n_estimators=param[0], max_depth=param[1])
    model.fit(X_train2, y_train2)
    pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, pred, squared=False)
    # rmse 기준으로 최소값을 찾는 알고리즘 적용
    if rmse < best_rmse:
        print(f'{param} 최적 값 갱신')
        best_rmse = rmse
        best_param = param
print('서칭 완료')

(70, 13) 서칭중
(70, 13) 최적 값 갱신
(70, 14) 서칭중
(70, 14) 최적 값 갱신
(70, 15) 서칭중
(70, 15) 최적 값 갱신
(70, 16) 서칭중
(70, 16) 최적 값 갱신
(70, 17) 서칭중
(70, 18) 서칭중
(70, 18) 최적 값 갱신
(80, 13) 서칭중
(80, 14) 서칭중
(80, 15) 서칭중
(80, 15) 최적 값 갱신
(80, 16) 서칭중
(80, 17) 서칭중
(80, 18) 서칭중
(90, 13) 서칭중
(90, 14) 서칭중
(90, 14) 최적 값 갱신
(90, 15) 서칭중
(90, 15) 최적 값 갱신
(90, 16) 서칭중
(90, 17) 서칭중
(90, 18) 서칭중
(100, 13) 서칭중
(100, 14) 서칭중
(100, 15) 서칭중
(100, 16) 서칭중
(100, 17) 서칭중
(100, 18) 서칭중
(120, 13) 서칭중
(120, 14) 서칭중
(120, 15) 서칭중
(120, 16) 서칭중
(120, 17) 서칭중
(120, 18) 서칭중
(140, 13) 서칭중
(140, 14) 서칭중
(140, 15) 서칭중
(140, 16) 서칭중
(140, 17) 서칭중
(140, 18) 서칭중
(160, 13) 서칭중
(160, 14) 서칭중
(160, 15) 서칭중
(160, 16) 서칭중
(160, 17) 서칭중
(160, 18) 서칭중
서칭 완료


In [20]:
# 위 파라메터 서칭 코드로 찾은 최적 모델의 평가지표와 파라메터 확인
best_rmse, best_param

(4.825947372289556, (90, 15))

In [17]:
from sklearn.metrics import r2_score, mean_squared_error

In [21]:
# 최적 모델로 모델 다시 학습 및 평가
best_model = RandomForestRegressor(random_state=42, n_estimators=best_param[0], max_depth=best_param[1])
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
print(r2_score(y_test, best_pred))
print(mean_squared_error(y_test, best_pred, squared=False))

0.8685953418406718
3.129116145413737


In [ ]:
'''dtr
0.8523871394649085
3.316487778373004

0.869257675459985
3.1212201657709477

0.8685953418406718
3.129116145413737

'''

## sklearn GridSearchCV
sklearn 패키지에는 위의 파라메터 서칭 과정을 간편하게 진행 할 수 있도록 GridSearchCV 방법론을 제공합니다.  
기존 파라메터 서칭과 함께 cross validation 과정을 추가하여 데이터 분할에 강건한 모델을 선택할 수 있도록 제작 되었습니다.

In [ ]:
''' xgboost 파라메터 서칭 목록
max_depth : classification [3, 5, 7, 9], regression [7, 9, 11, 13, 15]
n_estimators : 부스팅 트리 갯수, [100, 300, 500, 700, 1000, 2000, 3000]
learning_rate : round별 학습률 [0.001, 0.003, 0.01, 0.03]
subsample : 부트스트랩 샘플 비율 [0.6, 0.7, 0.8]
colsample_bytree : 부트스트랩 컬럼 비율 [0.6, 0.7, 0.8]
설명변수 갯수가 많은 경우 아래 파라메터도 서칭
reg_alpha : L1, lasso [1, 3, 5, 7, 9]
reg_lambda : L2, ridge, [1, 3, 5, 7, 9]
'''

In [ ]:
# 그리드 서치 import


In [ ]:
# 그리드 서치를 위한 모델 및 파라메터 준비


In [ ]:
# 그리드 서치 모델 생성

'''
estimator : 모델 딕셔너리
param_grid : 파라메터 딕셔너리
scoring=None : 평가방법
n_jobs=None : 학습에 사용할 컴퓨터 코어 갯수
verbose=0 : 리포트 형식 0, 1, 2

scoring 참고
https://scikit-learn.org/stable/modules/model_evaluation.html
'''

In [ ]:
# grid 학습


In [ ]:
# 최적 모델 및 파라메터 확인


In [ ]:
# 최적 모델로 모델 다시 학습 및 평가
